# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState
import random

player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=random.randint(0, 2),
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="던전을 돌면 뭐가 좋다 했지?")],    
    "dungenon_player": player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
await graph.ainvoke({
    "messages":[add_human_message("스킬 사용법이 뭐야?")]
}, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("내 아이템중에 가장 쎈 무기가 뭐야?")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: W

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
check_prompt:: <Abilities Used>
['USAGE_GUIDE']
</Abilities Used>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 1,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <사용 방법·조작 안내>

# 게임 시스템 개요

본 문서는 던전 탐사 중심의 게임 플레이 루프, 캐릭터 조작 방안,  
그리고 히로인·현자 NPC와의 상호작용 시스템을 포함한 주요 게임 메커니즘을 정리한 내용입니다.

---

# 1. 게임 핵심 순환 구조

게임은 다음과 같은 순환 구조를 기반으로 진행됩니다.

## 1-1. 던전 탐사
- 플레이어는 **로그라이크 방식의 던전**을 탐험합니다.
- 전투를 통해 스킬을 사용하고, 무기 및 장비를 강화하며 파밍을 진행합니다.

## 1-2. 경험치 및 기억의 조각 획득
- 탐험 과정에서 다양한 자원을 수집하고 경험치를 획득합니다.
- 전투와 선택 결과가 누적되어 **기억의 조각**이 생성됩니다.

## 1-3. 상담 및 대화 (

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-06T18:10:17.813762'}, response_metadata={}, id='29f6d9bf-c5da-4390-8307-e93b6c4d43be'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:19.080615', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='8132aa0c-3df6-44c7-aca0-b54d3e7e5d4d'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:19.081844'}, response_metadata={}, id='6e145a08-e435-49e9-ba98-1347fb4e7602'),
 AIMessage(content='스킬 사용법은 키보드의 E 키를 눌러서 무기 스킬을 사용하고, R 키를 눌러서 직업 고유 스킬을 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:20.038145', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='2ab69180-7713-438f-b316-319f3431b213'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:20.039419'}, response_metadata={}, id='5654c44d-7de6-418a-9a1d-8b3461e0b8ef'),
 AIMessage(content=

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [2]:
await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?")}, config=config
)

player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=random.randint(0, 2),
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="내가 잡을 수 있지?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.MONSTER_GUIDE: 'MONSTER_GUIDE'>]
check_prompt:: <Abilities Used>
['MONSTER_GUIDE']
</Abilities Used>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 1,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <몬스터 정보>
[{'monsterId': 0, 'monsterType': 0, 'monsterName': '스켈레톤', 'hp': 300, 'speed': 350, 'attack': 10, 'attackSpeed': 1, 'attackRange': 100, 'staggerGage': 10}, {'monsterId': 1, 'monsterType': 0, 'monsterName': '슬라임', 'hp': 250, 'speed': 200, 'attack': 10, 'attackSpeed': 1, 'attackRange': 100, 'staggerGage': 5}]
    </몬스터 정보>
</Current Situation>

While following all rul

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-06T18:10:17.813762'}, response_metadata={}, id='29f6d9bf-c5da-4390-8307-e93b6c4d43be'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:19.080615', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='8132aa0c-3df6-44c7-aca0-b54d3e7e5d4d'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:19.081844'}, response_metadata={}, id='6e145a08-e435-49e9-ba98-1347fb4e7602'),
 AIMessage(content='스킬 사용법은 키보드의 E 키를 눌러서 무기 스킬을 사용하고, R 키를 눌러서 직업 고유 스킬을 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:20.038145', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='2ab69180-7713-438f-b316-319f3431b213'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:20.039419'}, response_metadata={}, id='5654c44d-7de6-418a-9a1d-8b3461e0b8ef'),
 AIMessage(content=

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [3]:
player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=2,
    difficulty=random.randint(0, 2),
    hp=250,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
check_prompt:: <Abilities Used>
['EVENT_GUIDE']
</Abilities Used>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 2,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <이벤트>
{'room_id': 3, 'event_type': 0, 'event_title': '미치광이 상인', 'event_code': 'MAD_MERCHANT', 'scenario_text': '방의 한 가운데에 낡은 후드를 쓴 사람이 서 있다. 자신을 상인이라고 소개한 그는, 기괴한 웃음을 멈추지 못하며 이쪽을 뚫어지게 응시한다. 그의 눈빛에는 광기와 탐욕이 뒤섞여 있다.', 'scenario_narrative': '축축한 돌바닥 위, 방 한가운데에 선 낡은 후드가 미세하게 떨린다. 후드 안에서 드러난 얼굴은 창백하게 일그러져 있고, 갈라진 입술 사이로 끊임없는 키득거림이 새어 나온다. 상인은 자신을 상인이라 반복해 중얼거리며, 시선 한 번 흐트러뜨리지 않고 망각자를 꿰

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-06T18:10:17.813762'}, response_metadata={}, id='29f6d9bf-c5da-4390-8307-e93b6c4d43be'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:19.080615', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='8132aa0c-3df6-44c7-aca0-b54d3e7e5d4d'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:19.081844'}, response_metadata={}, id='6e145a08-e435-49e9-ba98-1347fb4e7602'),
 AIMessage(content='스킬 사용법은 키보드의 E 키를 눌러서 무기 스킬을 사용하고, R 키를 눌러서 직업 고유 스킬을 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:20.038145', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='2ab69180-7713-438f-b316-319f3431b213'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:20.039419'}, response_metadata={}, id='5654c44d-7de6-418a-9a1d-8b3461e0b8ef'),
 AIMessage(content=

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [4]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="다음 방은 뭐가 나올까?") },
    config=config
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>]
check_prompt:: <Abilities Used>
['DUNGEON_NAVIGATOR']
</Abilities Used>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 2,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <던전 안내>
        <던전맵>

아래는 현재 플레이어가 탐험 중인 던전의 전체 구조를 설명하는 데이터입니다.
던전은 방(room), 몬스터 스폰 정보, 보상 테이블 등으로 구성되며,
각 필드의 의미와 값의 범위는 다음 스펙을 참고하면 됩니다.

이 정보는 현재 위치한 방의 형태, 연결된 경로, 등장 가능한 몬스터,
획득 가능한 보상 등을 이해하는 데 사용됩니다.

■ playerIds (int[])
  - 던전에 참여한 플레이어 ID 목록

■ heroineIds (int[])
  - 각 플레이어가 선택한 히로인 ID 목록

■ rooms (roomData[])
  던전을 구성하는 방 목록입니다.

  ● roomId (int)
    

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-06T18:10:17.813762'}, response_metadata={}, id='29f6d9bf-c5da-4390-8307-e93b6c4d43be'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:19.080615', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='8132aa0c-3df6-44c7-aca0-b54d3e7e5d4d'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:19.081844'}, response_metadata={}, id='6e145a08-e435-49e9-ba98-1347fb4e7602'),
 AIMessage(content='스킬 사용법은 키보드의 E 키를 눌러서 무기 스킬을 사용하고, R 키를 눌러서 직업 고유 스킬을 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:20.038145', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='2ab69180-7713-438f-b316-319f3431b213'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:20.039419'}, response_metadata={}, id='5654c44d-7de6-418a-9a1d-8b3461e0b8ef'),
 AIMessage(content=

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [5]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?") },
    config=config
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
check_prompt:: <Abilities Used>
['USAGE_GUIDE']
</Abilities Used>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 2,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <사용 방법·조작 안내>

# 게임 시스템 개요

본 문서는 던전 탐사 중심의 게임 플레이 루프, 캐릭터 조작 방안,  
그리고 히로인·현자 NPC와의 상호작용 시스템을 포함한 주요 게임 메커니즘을 정리한 내용입니다.

---

# 1. 게임 핵심 순환 구조

게임은 다음과 같은 순환 구조를 기반으로 진행됩니다.

## 1-1. 던전 탐사
- 플레이어는 **로그라이크 방식의 던전**을 탐험합니다.
- 전투를 통해 스킬을 사용하고, 무기 및 장비를 강화하며 파밍을 진행합니다.

## 1-2. 경험치 및 기억의 조각 획득
- 탐험 과정에서 다양한 자원을 수집하고 경험치를 획득합니다.
- 전투와 선택 결과가 누적되어 **기억의 조각**이 생성됩니다.

## 1-3. 상담 및 대화 (

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-06T18:10:17.813762'}, response_metadata={}, id='29f6d9bf-c5da-4390-8307-e93b6c4d43be'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:19.080615', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='8132aa0c-3df6-44c7-aca0-b54d3e7e5d4d'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:19.081844'}, response_metadata={}, id='6e145a08-e435-49e9-ba98-1347fb4e7602'),
 AIMessage(content='스킬 사용법은 키보드의 E 키를 눌러서 무기 스킬을 사용하고, R 키를 눌러서 직업 고유 스킬을 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-06T18:10:20.038145', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='2ab69180-7713-438f-b316-319f3431b213'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-06T18:10:20.039419'}, response_metadata={}, id='5654c44d-7de6-418a-9a1d-8b3461e0b8ef'),
 AIMessage(content=

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
await graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는법 힌트좀 줘") },
    config=config
)
result = await graph.ainvoke(
    { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
    config=config
)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instanc

[HumanMessage(content='히로인의 호감도 얻는법 힌트좀 줘', additional_kwargs={'created_at': '2025-12-06T23:25:24.309566'}, response_metadata={}, id='1fdfebd1-3555-4c3d-896f-abaab3e2e79d'),
 AIMessage(content='로코는 따뜻한 열기와 머리 쓰다듬는 걸 좋아해요! 금속 소리도 반응 좋고, 정직함이 최고예요~ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 2967, 'total_tokens': 3371, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 362, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 2967, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': '5c5821f1-c610-ddfc-83e9-4d9dad9fe679', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--351cccb3-557d-4190-a1f9-ad53b4182794-0', usage_metadata={'input_tokens': 2967, 'output_tokens': 42, 'total_tokens': 3371, 'input_